In [5]:
from __future__ import division
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
import pandas as pd
import math
from numpy import random
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import plot_confusion_matrix
import sys; sys.path.append('./')
from utilssln import Utils2

In [6]:
ut = Utils2()

In [7]:
dataset_rfe = '../../../data/CTGsmtRFE.csv'
X, Y = ut.get_matrixes(dataset_rfe)

In [8]:
def train_svm(kernel, C, gamma):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = []
    EficienciaVal = []
    skf = StratifiedKFold(n_splits=Folds)
    porc_vectores = []
    f1 = []
    gmean = []
    models = []
    for train, test in skf.split(X, Y):
        Xtrain, Ytrain, Xtest, Ytest = ut.get_training_test(X, Y, train, test)
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        model = SVC(C=C,kernel=kernel, gamma=gamma, probability=True)
        model.fit(Xtrain, Ytrain)
        models.append(model)
        #Validación
        f1_score_current, gmean_current, eficiencia_train_current, eficiencia_test_current = ut.get_metrics(model=model, X_train=Xtrain, X_test=Xtest, y_train=Ytrain, Ytest=Ytest)
        gmean.append(gmean_current)
        f1.append(f1_score_current)
        EficienciaTrain.append(eficiencia_train_current)
        EficienciaVal.append(eficiencia_test_current)
    
    f1_m = np.mean(f1)
    f1_ic = np.std(f1)
    gmean = np.mean(gmean)
    gmean_ic = np.std(gmean)
    efi_train = np.mean(EficienciaTrain)
    efi_train_ic = np.std(EficienciaTrain)
    efi_test = np.mean(EficienciaVal)
    efi_test_ic = np.std(EficienciaVal)
        
    print(f"Finalizado: {kernel} C: {C} gamma: {gamma}")
    
    return f1_m, f1_ic, gmean, gmean_ic, efi_train, efi_train_ic, efi_test, efi_test_ic